# Objetivo

Este trabalho tem como objetivo aplicar práticas de engenharia de dados aprendidas no curso e constuir um pipeline completo de dados utilizando uma plataforma de nuvem. 

Foram selecionados dados históricos de partidas de tênis profissional, um assunto que me interessa e a partir disso foram elaboradas questões sobre as conquistas dos jogadores. 

Perguntas:
1. Quantos jogadores já ganharam os 4 grand slams? ("Career Grand Slam")
2. Quantos já conseguiram os 4 grand slams no mesmo ano e qual foi o ultimo ano que isso ocorreu?
3. Que jogador permaneceu mais tempo no top 1? E no top 5?
4. Que caracteristicas em comum tem essses jogadores?

# Coleta

## Coleta do dataset

A coleta dos dados utilizados neste trabalho é realizada a partir do **repositório do Kaggle**, por meio da API oficial da plataforma. Para garantir uma autenticação segura e automatizada dentro do ambiente do **Databricks**, foi necessário criar um processo específico para carregar as variáveis de ambiente contendo as credenciais da API do Kaggle.

Como o Databricks não consegue acessar diretamente os arquivos `.env` armazenados no DBFS de forma nativa, foi implementada uma função personalizada (`load_env_from_dbfs`) para ler e interpretar o conteúdo do arquivo `.env` como texto. Essa abordagem se mostrou eficaz para popular as variáveis de ambiente dentro da sessão ativa do notebook.

Após a configuração do ambiente, o dataset **"tennis"** foi baixado do Kaggle e descompactado no diretório temporário `/tmp`. O arquivo principal, um banco de dados no formato **SQLite**, foi então carregado e inspecionado. As tabelas disponíveis — `matches`, `players` e `rankings` — foram extraídas utilizando consultas SQL diretas, convertidas para DataFrames do Pandas e posteriormente carregadas para o DBFS utilizando spark.


## Setup do ambiente
- Install de bibliotecas
- Set de variaveis de ambientes do .env (Foi necessário para não deixar as credenciais do kaggle publicadas no código).

In [0]:
# Instalar a biblioteca do Kaggle (necessário sempre que reinicia o cluster, alternativa seria botar um initscript no cluster)
!pip install kaggle
!pip install python-dotenv

import pandas as pd
import io
import requests
import sqlite3
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta



     |████████████████████████████████| 173 kB 6.7 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 78 kB 9.2 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-cd699898-eee6-408d-a562-6c4f219649df/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-cd699898-eee6-408d-a562-6c4f219649df/bin/python -m pip install --upgrade pip' command.


In [0]:
# Função para conseguir acessar o .env, ela lê o conteúdo do arquivo como uma string inteira, diretamente da interface DBFS da Databricks
# Foi a unica solução encontrada uma vez que o databricks não conseguiu ler diretamente o arquivo do DBFS.

def load_env_from_dbfs(path: str):
    env_text = dbutils.fs.head(path)
    for line in io.StringIO(env_text):
        if "=" in line:
            key, value = line.strip().split("=", 1)
            os.environ[key] = value

load_env_from_dbfs("dbfs:/FileStore/env/.env")

# Caminho local para acessar arquivo que contem as variaveis de o DBFS
env_path = "/dbfs/FileStore/env/.env"

# Carrega as variáveis de ambiente
load_dotenv(dotenv_path=env_path, override=True)

# Defina suas credenciais do Kaggle (caso queira rodar coloque explicitamente no codigo ou carregue um arquivo .env no dbfs e ajuste o caminho acima)
os.environ["KAGGLE_USERNAME"] = os.getenv("KAGGLE_USERNAME")             # Coloque o seu nome de usuário do Kaggle aqui 
os.environ["KAGGLE_KEY"] = os.getenv("KAGGLE_KEY")                       # Coloque a sua chave API do Kaggle aqui



In [0]:
# Baixar o dataset do Kaggle
!kaggle datasets download -d guillemservera/tennis -p /tmp --unzip

# Listar arquivos baixados para verificar
os.listdir('/tmp')


Dataset URL: https://www.kaggle.com/datasets/guillemservera/tennis
License(s): Attribution-NonCommercial 4.0 International (CC BY-NC 4.0)
Out[62]: ['hsperfdata_root',
 'driver-env.sh',
 'systemd-private-1b1b47cc845b43759a67a2c95d8db58a-systemd-resolved.service-9SoJFg',
 '.ICE-unix',
 '.X11-unix',
 'chauffeur-env.sh',
 'chauffeur-daemon-params',
 'systemd-private-1b1b47cc845b43759a67a2c95d8db58a-apache2.service-BDKDqh',
 'tmp.XmdLHBNBJ3',
 'driver-daemon-params',
 'custom-spark.conf',
 'database.sqlite',
 'chauffeur-daemon.pid',
 'tennis_atp',
 'RtmpLHDb5M',
 '.Test-unix',
 'Rserv',
 '.XIM-unix',
 'systemd-private-1b1b47cc845b43759a67a2c95d8db58a-systemd-logind.service-WjlJRe',
 'driver-daemon.pid',
 '.font-unix',
 'systemd-private-1b1b47cc845b43759a67a2c95d8db58a-ntp.service-cS4vVg']

In [0]:

# Definir o caminho do arquivo baixado
file_path = "/tmp/database.sqlite"  

# Conectar ao banco de dados SQLite
conn = sqlite3.connect(file_path)

# Listar tabelas disponíveis no banco de dados
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
print("Tabelas disponíveis:", tables)


Tabelas disponíveis:        name
0   matches
1   players
2  rankings


In [0]:
# 
df_matches = pd.read_sql("SELECT * FROM matches", conn)
df_players = pd.read_sql("SELECT * FROM players", conn)
df_rankings = pd.read_sql("SELECT * FROM rankings", conn)

In [0]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947720 entries, 0 to 947719
Data columns (total 81 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   tourney_id           945297 non-null  object 
 1   tourney_name         947720 non-null  object 
 2   surface              941327 non-null  object 
 3   draw_size            946885 non-null  object 
 4   tourney_level        947720 non-null  object 
 5   tourney_date         944748 non-null  float64
 6   match_num            947720 non-null  object 
 7   winner_id            921321 non-null  float64
 8   winner_seed          404551 non-null  object 
 9   winner_entry         175563 non-null  object 
 10  winner_name          921321 non-null  object 
 11  winner_hand          901856 non-null  object 
 12  winner_ht            554864 non-null  float64
 13  winner_ioc           921195 non-null  object 
 14  winner_age           907669 non-null  float64
 15  loser_id         

In [0]:
# Excluindo apenas essa coluna pois de cara todos seus valores são nulos (estava dando problema no SELECT)
df_matches.drop("loser2_rank_points", axis=1, inplace=True)

## Camada Raw/Bronze
- Captura e persistência dos dados crus (raw)
- Sem transformação, schema original, histórico

Depois de criadas as tabelas dentro do database no DBFS, basta rodar o código do notebook auxiliar para trazer para memória do hive_metastore.

In [0]:
# %sql DROP DATABASE raw CASCADE

In [0]:
%sql
--Utilizado apenas a primeira vez para criação do database, depois carregado do DBFS
CREATE DATABASE raw;

In [0]:
# Necessário criar as tabelas apenas uma vez, depois de criadas no DBFS rodar o código do notebook auxiliar, para trazer para memória


matches_spark_df = spark.createDataFrame(df_matches)
matches_spark_df.write.mode("overwrite").saveAsTable("raw.matches")

players_spark_df = spark.createDataFrame(df_players)
players_spark_df.write.mode("overwrite").saveAsTable("raw.players")

rankings_spark_df = spark.createDataFrame(df_rankings)
rankings_spark_df.write.mode("overwrite").saveAsTable("raw.rankings")


In [0]:
%sql SELECT * FROM raw.matches LIMIT 10

tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner1_id,winner2_id,loser1_id,loser2_id,winner1_name,winner1_hand,winner1_ht,winner1_ioc,winner1_age,winner2_name,winner2_hand,winner2_ht,winner2_ioc,winner2_age,loser1_name,loser1_hand,loser1_ht,loser1_ioc,loser1_age,loser2_name,loser2_hand,loser2_ht,loser2_ioc,loser2_age,winner1_rank,winner1_rank_points,winner2_rank,winner2_rank_points,loser1_rank,loser1_rank_points,loser2_rank
2018-M020,Brisbane,Hard,32,A,2.0180101E7,271,105992.0,null,null,Ryan Harrison,R,183.0,USA,25.6,104919.0,null,null,Leonardo Mayer,R,188.0,ARG,30.6,6-4 3-6 6-2,3,R32,123.0,9.0,2.0,82.0,49.0,39.0,20.0,13.0,8.0,9.0,10.0,3.0,80.0,47.0,33.0,19.0,14.0,1.0,4.0,47.0,1010.0,52.0,909.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2018-M020,Brisbane,Hard,32,A,2.0180101E7,272,111577.0,null,null,Jared Donaldson,R,188.0,USA,21.2,111442.0,null,WC,Jordan Thompson,R,183.0,AUS,23.7,6-2 6-4,3,R32,90.0,5.0,3.0,58.0,32.0,25.0,14.0,9.0,4.0,5.0,3.0,5.0,62.0,41.0,25.0,7.0,9.0,7.0,11.0,54.0,890.0,94.0,593.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2018-M020,Brisbane,Hard,32,A,2.0180101E7,273,104797.0,null,null,Denis Istomin,R,188.0,UZB,31.3,106000.0,7.0,null,Damir Dzumhur,R,172.0,BIH,25.6,6-7(4) 6-3 6-2,3,R32,145.0,7.0,0.0,94.0,66.0,48.0,12.0,14.0,9.0,11.0,8.0,6.0,120.0,53.0,37.0,29.0,15.0,10.0,16.0,63.0,809.0,30.0,1391.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2018-M020,Brisbane,Hard,32,A,2.0180101E7,275,200282.0,null,WC,Alex De Minaur,R,183.0,AUS,18.8,105449.0,null,null,Steve Johnson,R,188.0,USA,28.0,7-6(7) 6-4,3,R32,104.0,9.0,3.0,66.0,37.0,32.0,17.0,11.0,2.0,3.0,6.0,2.0,80.0,43.0,33.0,17.0,11.0,4.0,6.0,208.0,245.0,44.0,1055.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2018-M020,Brisbane,Hard,32,A,2.0180101E7,276,111581.0,null,Q,Michael Mmoh,R,188.0,USA,19.9,105643.0,null,null,Federico Delbonis,L,190.0,ARG,27.2,6-3 6-4,3,R32,69.0,5.0,4.0,55.0,38.0,32.0,11.0,10.0,3.0,3.0,4.0,0.0,45.0,35.0,28.0,5.0,9.0,0.0,2.0,175.0,299.0,68.0,755.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2018-M020,Brisbane,Hard,32,A,2.0180101E7,277,104999.0,8.0,null,Mischa Zverev,L,190.0,GER,30.3,105441.0,null,Q,John Patrick Smith,L,188.0,AUS,28.9,6-4 7-5,3,R32,88.0,4.0,1.0,66.0,51.0,35.0,6.0,11.0,3.0,6.0,4.0,0.0,69.0,51.0,27.0,8.0,11.0,1.0,6.0,33.0,1302.0,218.0,235.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2018-M020,Brisbane,Hard,32,A,2.0180101E7,278,105238.0,null,null,Alexandr Dolgopolov,R,180.0,UKR,29.1,106043.0,6.0,null,Diego Schwartzman,R,170.0,ARG,25.3,6-4 7-6(3),3,R32,93.0,6.0,4.0,69.0,40.0,32.0,16.0,11.0,3.0,4.0,1.0,2.0,70.0,34.0,22.0,20.0,11.0,1.0,3.0,38.0,1231.0,26.0,1675.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2018-M020,Brisbane,Hard,32,A,2.0180101E7,279,104547.0,null,null,Horacio Zeballos,L,188.0,ARG,32.6,124014.0,null,Q,Ernesto Escobedo

In [0]:
%sql SELECT * FROM raw.players LIMIT 10

player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
100001,Gardnar,Mulloy,R,1.9131122E7,USA,185.0,Q54544
100002,Pancho,Segura,R,1.921062E7,ECU,168.0,Q54581
100003,Frank,Sedgman,R,1.9271002E7,AUS,180.0,Q962049
100004,Giuseppe,Merlo,R,1.9271011E7,ITA,null,Q1258752
100005,Richard,Gonzalez,R,1.9280509E7,USA,188.0,Q53554
100006,Grant,Golden,R,1.9290821E7,USA,175.0,Q3115390
100007,Abe,Segal,L,1.9301023E7,RSA,null,Q1258527
100008,Kurt,Nielsen,R,1.9301119E7,DEN,null,Q552261
100009,Istvan,Gulyas,R,1.9311014E7,HUN,null,Q51066
100010,Luis,Ayala,R,1.9320918E7,CHI,170.0,Q1275397


In [0]:
%sql SELECT * FROM raw.rankings LIMIT 10

ranking_date,rank,player,points
20000110,1,101736,4135.0
20000110,2,102338,2915.0
20000110,3,101948,2419.0
20000110,4,103017,2184.0
20000110,5,102856,2169.0
20000110,6,102358,2107.0
20000110,7,102839,1966.0
20000110,8,101774,1929.0
20000110,9,102701,1846.0
20000110,10,101990,1739.0


# Modelagem
## Modelo Estrela

### Tabela Fato
- **f_Matches**: contém o resultado e estatísticas das partidas
  - FK: `tourney_id`, `winner_id`, `loser_id`, `tourney_date`
  - Métricas: aces, double faults, pontos de saque, tempo, score, ranking etc.

### Tabelas Dimensão
- **d_Players**: dados dos jogadores (mão, altura, país, data de nascimento)
- **d_Tournaments**: informações dos torneios (superfície, nível, tamanho da chave)
- **d_Calendario**: tempo em granularidade de dia, mês, ano
- **d_Ranking**: posição e pontos de ranking por jogador por data (1 data por semana)


Motivos para a seleção do modelo estrela:
- Nosso objetivo é análise rápida e clara de dados históricos de partidas, rankings e torneios.
- As dimensões são simples e não precisam ser quebradas.
- Ganhamos em performance e simplicidade.
- É o modelo mais usado em BI

Esse modelo poderia evoluir para um modelo Snow Flake, porém o Snow Flake:
- Pode deixar o modelo mais organizado, mas também mais complicado.
- Requer mais joins, o que pode deixar as consultas mais lentas.
- Indicado quando há hierarquias complexas (ex: país → estado → cidade).


# Tranformação e Carga

## Camada Trusted/Silver
- Limpeza e data quality
- Dados consistentes e confiáveis

In [0]:
%sql CREATE DATABASE trusted;

### Qualidade de Dados

Nesta seção, foram abordadas algumas das tratativas de dados identificadas em um primeiro momento. Essas tratativas foram aplicadas nas tabelas *raw* de **players**, **rankings** e **matches** para garantir a integridade, consistência e precisão das informações.


### Tabela de jogadores (players)


Existem ocorrencias na tabela de jogadores com nome e sobrenome duplicados e que possuem a mesma data de nascimento, como correção devem ser removidas as duplicatas, porém é necessário verificar o impacto disso nas outras tabelas.

Como solução estou criando uma nova camada de staging para criar uma tabela que vai mapear o id que será mantido (canonical_player_id), arbitrariamente escolhi o menor id para ser mantido. Como solução mais rebuscada poderia selecionar os ids com campos mais corretos (ex.: menos campos nulos, maior altura, mão dominante diferente de U).

O objetivo dessa staging é utilizar nas demais tabelas para alterar os ids que foram removidos da tabela players e botar os ids que ficaram.

Por fim, ao criar a tabela players na camada trusted sem duplicatas, foi removida a coluna "wikidata_id", pois não tinha utilidade.

In [0]:
%sql
SELECT 
    name_first, 
    name_last, 
    dob,
    COUNT(*) as count_duplicados
FROM 
    raw.players
GROUP BY 
    name_first, 
    name_last,
    dob
HAVING 
    COUNT(*) > 1;


name_first,name_last,dob,count_duplicados
Kenichi,Kiyomiya,1.9671101E7,2
Juan,Jimenez,null,2
Attila,Korpas,1.9450901E7,2
P,Quilichini,null,2
Phillip,Williamson,1.9650529E7,2
Jose,Tort,null,2
Fabiano,Mintz,1.9750516E7,2
Jean Pierre,Olivier,null,2
Gaston,Naon,1.9700925E7,2
Kurt,Robinson,1.9660301E7,2


In [0]:
%sql
select * from raw.players where name_first= "Dan" and name_last ="Martin"

player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
206876,Dan,Martin,U,1.9990509E7,CAN,null,null
209399,Dan,Martin,U,1.9990509E7,CAN,null,null
202297,Dan,Martin,U,1.9990509E7,CAN,null,null


In [0]:
%sql CREATE DATABASE staging;

In [0]:
%sql
drop table staging.player_id_mapping

In [0]:
%sql
CREATE TABLE staging.player_id_mapping AS
WITH jogadores_duplicados AS (
    SELECT
        name_first,
        name_last,
        dob,
        MIN(player_id) AS canonical_player_id
    FROM raw.players
    GROUP BY name_first, name_last, dob
    HAVING COUNT(*) > 1
)
SELECT 
    p.player_id AS duplicate_player_id,
    jd.canonical_player_id,
    p.name_first,
    p.name_last,
    p.dob
FROM raw.players p
JOIN jogadores_duplicados jd
  ON p.name_first = jd.name_first
 AND p.name_last = jd.name_last
 AND p.dob = jd.dob
WHERE p.player_id != jd.canonical_player_id;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Exemplo de caso duplicado e como ele é utilizado na player_id_mapping, esse caso possuia 3 ids diferentes, e sera mantido apenas o canonical_player_id (202297)

SELECT * FROM staging.player_id_mapping WHERE name_first = "Dan"

duplicate_player_id,canonical_player_id,name_first,name_last,dob
209399,202297,Dan,Martin,1.9990509E7
206876,202297,Dan,Martin,1.9990509E7


In [0]:
%sql
CREATE TABLE trusted.players AS
WITH
-- IDs duplicados que devem ser ignorados
ids_duplicados AS (
    SELECT duplicate_player_id FROM staging.player_id_mapping
),
-- Dados consolidados (sem a coluna wikidata_id)
consolidados AS (
    SELECT 
        player_id,
        name_first,
        name_last,
        hand,
        dob,
        ioc,
        height
    FROM raw.players
    WHERE player_id NOT IN (SELECT duplicate_player_id FROM ids_duplicados)
)
SELECT * FROM consolidados;


num_affected_rows,num_inserted_rows


### Tabela de rank (rankings)


Também existem ocorrencias na tabela de rankings com jogadores duplicados para a mesma data, porém um jogador deveria ter 2 posições no rank no mesmo dia.

Como solução arbitrariamente escolhi o menor rank no dia será mantido, ou seja, o momento em que ele tem o maior numero de pontos registrados no dia.

Outro tratamento nessa tabela é utilizar a tabela de mapeamento de ids criada na staging para alterar os ids que foram removidos da tabela players e botar os ids que ficaram.

In [0]:
%sql
SELECT ranking_date, player, COUNT(*)
FROM raw.rankings
GROUP BY ranking_date, player
HAVING COUNT(*) > 1;


ranking_date,player,count(1)
20220523,208147,2
20210823,208142,2
20210719,144703,2
20210809,208147,2
20220207,208142,2
20211004,208147,2
20210927,206904,2
20211018,206904,2
20210719,127032,2
20231225,208147,2


In [0]:
%sql
CREATE TABLE trusted.rankings AS
WITH rankings_com_ids_corrigidos AS (
    SELECT
        r.ranking_date,
        r.rank,
        COALESCE(m.canonical_player_id, r.player) AS player,  -- substitui o ID se houver mapeamento
        r.points
    FROM raw.rankings r
    LEFT JOIN staging.player_id_mapping m
      ON r.player = m.duplicate_player_id
),
rankings_deduplicados AS (
    SELECT *
    FROM (
        SELECT *,
               ROW_NUMBER() OVER (
                   PARTITION BY ranking_date, player
                   ORDER BY rank ASC
               ) AS rn
        FROM rankings_com_ids_corrigidos
    ) sub
    WHERE rn = 1
)
SELECT 
    ranking_date,
    rank,
    player,
    points
FROM rankings_deduplicados;


num_affected_rows,num_inserted_rows


In [0]:
%sql --Exemplo que está sem duplicatas
SELECT ranking_date, player, COUNT(*)
FROM trusted.rankings
GROUP BY ranking_date, player
HAVING COUNT(*) > 1;

ranking_date,player,count(1)


### Tabela de partidas (matches)


Nessa tabela o principal tratamento foi aplicar a tabela de mapeamento de ids criada na staging para alterar os ids que foram removidos da tabela players e botar os ids que ficaram, tanto no winner_id quanto no loser_id.

In [0]:
%sql
CREATE TABLE trusted.matches AS
SELECT
    rm.* EXCEPT (winner_id, loser_id),
    COALESCE(mw.canonical_player_id, rm.winner_id) AS winner_id,
    COALESCE(ml.canonical_player_id, rm.loser_id) AS loser_id
FROM raw.matches rm
LEFT JOIN staging.player_id_mapping mw
  ON rm.winner_id = mw.duplicate_player_id
LEFT JOIN staging.player_id_mapping ml
  ON rm.loser_id = ml.duplicate_player_id;


num_affected_rows,num_inserted_rows


## Camada Business/Gold
- Visão analítica pronta para consumo
- Aplicação do modelo estrela
- Otimizado para negócio, agregações e métricas

In [0]:
%sql
-- Criar schema se ainda não existir (obs. create schema e create database da no mesmo para o interpretador do databricks)
CREATE SCHEMA IF NOT EXISTS business;

In [0]:
%sql
-- Dimensão: Jogadores
CREATE TABLE business.d_players AS
SELECT DISTINCT
    player_id,
    concat(name_first," ",name_last) AS player_name,
    hand,
    height,
    ioc,
    dob AS date_of_birth
FROM trusted.players;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Dimensão: Rankings
CREATE TABLE business.d_ranking AS
SELECT
    player,
    ranking_date,
    rank,
    points
FROM trusted.rankings;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Dimensão: Calendario
drop table business.d_calendario;
CREATE TABLE business.d_calendario (
    id_data INT,
    data DATE,
    ano INT,
    mes INT,
    nome_mes STRING
)


In [0]:
%sql
-- Inserção dos dados com geração de datas via SEQUENCE
INSERT INTO business.d_calendario
SELECT 
    CAST(date_format(data, 'yyyyMMdd') AS INT) AS id_data,
    data,
    YEAR(data) AS ano,
    MONTH(data) AS mes,
    DATE_FORMAT(data, 'MMMM') AS nome_mes
FROM (
    SELECT explode(sequence(DATE('1900-01-01'), current_date(), interval 1 day)) AS data
);

num_affected_rows,num_inserted_rows
45757,45757


In [0]:
%sql
-- Dimensão: Torneios
CREATE TABLE business.d_tournaments AS
SELECT DISTINCT
    tourney_id,
    tourney_name,
    surface,
    draw_size,
    tourney_level
FROM trusted.matches;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE TABLE business.f_matches AS
SELECT
    match_num,
    tourney_id,
    tourney_date,
    winner_id,
    loser_id,
    best_of,
    round,
    score,
    minutes,

    -- Estatísticas do vencedor
    w_ace, w_df, w_svpt,
    w_1stIn, w_1stWon, w_2ndWon,
    w_SvGms, w_bpSaved, w_bpFaced,

    -- Estatísticas do perdedor
    l_ace, l_df, l_svpt,
    l_1stIn, l_1stWon, l_2ndWon,
    l_SvGms, l_bpSaved, l_bpFaced,

    -- Rankings
    winner_rank,
    winner_rank_points,
    loser_rank,
    loser_rank_points

FROM trusted.matches;

num_affected_rows,num_inserted_rows


# Dicionário de Dados – Camada Business/Gold

**Pipeline dos dados**: `raw` (dados brutos) -> `trusted` (dados tratados) -> `business` (dados modelados)

**Origem dos dados**: camada `trusted`  
**Destino**: camada `business`  
**Padrão de nomenclatura**:  
- `d_` = tabelas dimensão  
- `f_` = tabelas fato

---

## 🔷 Tabela: `business.d_players` — Dimensão Jogadores

| Campo         | Tipo     | Descrição                              | Domínio / Exemplo               |
|---------------|----------|----------------------------------------|---------------------------------|
| `player_id`   | STRING   | Identificador único do jogador         | "101501", "200123"              |
| `player_name` | STRING   | Nome completo do jogador               | "Roger Federer"                 |
| `hand`        | CHAR(1)  | Mão dominante                          | "R" = destro, "L" = canhoto     |
| `height`      | INT      | Altura em centímetros                  | 180–210 cm                      |
| `ioc`         | STRING   | Código do país (ISO 3 letras)          | "SUI", "ESP", "USA"             |
| `date_of_birth` | INT   | Data de nascimento do jogador           | 19810818                    |

---

## 🔷 Tabela: `business.d_tournaments` — Dimensão Torneios

| Campo           | Tipo    | Descrição                                | Domínio / Exemplo                   |
|-----------------|---------|------------------------------------------|-------------------------------------|
| `tourney_id`    | STRING  | Identificador único do torneio           | "2021-888", "2019-030"              |
| `tourney_name`  | STRING  | Nome do torneio                          | "Wimbledon", "Roland Garros"        |
| `surface`       | STRING  | Tipo de quadra                           | "Clay", "Grass", "Hard"             |
| `draw_size`     | STRING  | Nº de jogadores no torneio               | "32", "64", "128"                   |
| `tourney_level` | STRING  | Nível do torneio                         | "G" (Grand Slam), "M", "A", "C"...  |

---

## 🔷 Tabela: `business.d_calendario` — Dimensão Calendario

| Campo         | Tipo   | Descrição                                | Exemplo            |
|---------------|--------|------------------------------------------|--------------------|
| `id_data`     | INT    | Data da semana do torneio                | 20210628           |
| `date   `     | DATE   | Data da semana do torneio                | "2021-06-28"       |
| `ano`         | INT    | Ano do torneio                           | 2021               |
| `mes`         | INT    | Mês do torneio                           | 6                  |
| `nome_mes`    | STRING | Nome do Mês do torneio                   | June               |

---

## 🔷 Tabela: `business.d_ranking` — Dimensão Ranking

| Campo             | Tipo   | Descrição                              | Exemplo                |
|-------------------|--------|----------------------------------------|------------------------|
| `player`          | INT    | ID do jogador                          | 101501                 |
| `ranking_date`    | INT    | FK → `d_calendario.id_data`            | 20210628               |
| `rank`            | INT    | Posição no ranking                     | 1, 25, 105             |
| `points`          | INT    | Pontos de ranking                      | 12000, 850, 55         |

---

##  🔷 Tabela: `business.f_matches` — Fato Partidas

| Campo                | Tipo     | Descrição                                 |
|----------------------|----------|-------------------------------------------|
| `match_id`           | STRING   | Identificador único da partida            |
| `tourney_id`         | STRING   | FK → `d_tournaments.tourney_id`           |
| `tourney_date`       | DATE     | FK → `d_calendario.id_data`               |
| `winner_id`          | STRING   | FK → `d_players.player_id` (vencedor)     |
| `loser_id`           | STRING   | FK → `d_players.player_id` (perdedor)     |
| `best_of`            | INT      | Número máximo de sets (3 ou 5)            |
| `round`              | STRING   | Fase do torneio (e.g. "QF", "SF", "F")     |
| `score`              | STRING   | Resultado textual do jogo                 |
| `minutes`            | INT      | Duração da partida (minutos)              |

###  Estatísticas do vencedor (`w_`)

| Campo       | Tipo | Descrição                       |
|-------------|------|---------------------------------|
| `w_ace`     | INT  | Aces                            |
| `w_df`      | INT  | Duplas faltas                   |
| `w_svpt`    | INT  | Pontos de saque                 |
| `w_1stIn`   | INT  | Primeiro saque dentro           |
| `w_1stWon`  | INT  | Pontos vencidos no 1º saque     |
| `w_2ndWon`  | INT  | Pontos vencidos no 2º saque     |
| `w_SvGms`   | INT  | Games de saque                  |
| `w_bpSaved` | INT  | Break points salvos             |
| `w_bpFaced` | INT  | Break points enfrentados        |

###  Estatísticas do perdedor (`l_`)

| Campo       | Tipo | Descrição                       |
|-------------|------|---------------------------------|
| `l_ace`     | INT  | Aces                            |
| `l_df`      | INT  | Duplas faltas                   |
| `l_svpt`    | INT  | Pontos de saque                 |
| `l_1stIn`   | INT  | Primeiro saque dentro           |
| `l_1stWon`  | INT  | Pontos vencidos no 1º saque     |
| `l_2ndWon`  | INT  | Pontos vencidos no 2º saque     |
| `l_SvGms`   | INT  | Games de saque                  |
| `l_bpSaved` | INT  | Break points salvos             |
| `l_bpFaced` | INT  | Break points enfrentados        |

###  Rankings

| Campo                 | Tipo | Descrição                             |
|-----------------------|------|---------------------------------------|
| `winner_rank`         | INT  | Ranking do vencedor                   |
| `winner_rank_points`  | INT  | Pontos de ranking do vencedor         |
| `loser_rank`          | INT  | Ranking do perdedor                   |
| `loser_rank_points`   | INT  | Pontos de ranking do perdedor         |


# Análise

## Solução do problema

### Pergunta 1
- Quantos jogadores já ganharam os 4 grand slams? ("Career Grand Slam")

R: 8 jogadores como podemos ver pela tabela abaixo.

Obs.: Quando fiz a query inicialmente estava faltando 1 jogador pelo meu conhecimento, pois estava considerando apenas 4 nomes diferentes dos torneios, depois percebi que tem um (US Open <> Us Open), por isso um deles tinha ganhado 5 grand slams diferentes, o que na teoria seria impossivel, esse caso deveria ter sido tratado na qualidade de dados, porém foi considerado >= 4 grand slams para trazer a resposta correta.

In [0]:
%sql
SELECT DISTINCT p.player_name
FROM (
  SELECT m.winner_id
  FROM business.f_matches m
  JOIN business.d_tournaments t 
    ON m.tourney_id = t.tourney_id
  WHERE t.tourney_level = 'G'
    AND m.round = 'F'
  GROUP BY m.winner_id
  HAVING COUNT(DISTINCT t.tourney_name) >= 4
) AS career_grand_slam
JOIN business.d_players p 
  ON career_grand_slam.winner_id = p.player_id;


player_name
Rafael Nadal
Roy Emerson
Novak Djokovic
Rod Laver
Don Budge
Fred Perry
Roger Federer
Andre Agassi


In [0]:
%sql
SELECT 
  p.player_name,
  t.tourney_name,
  t.tourney_level,
  m.round
FROM business.f_matches m
JOIN business.d_players p 
  ON m.winner_id = p.player_id
JOIN business.d_tournaments t 
  ON m.tourney_id = t.tourney_id
WHERE p.player_name = 'Novak Djokovic'
  AND m.round = 'F'
  AND T.tourney_level = 'G'
GROUP BY ALL 



player_name,tourney_name,tourney_level,round
Novak Djokovic,Wimbledon,G,F
Novak Djokovic,Roland Garros,G,F
Novak Djokovic,Australian Open,G,F
Novak Djokovic,US Open,G,F
Novak Djokovic,Us Open,G,F


### Pergunta 2
- Quantos já conseguiram os 4 grand slams no mesmo ano e qual foi o ultimo ano que isso ocorreu?

R: De fato apenas esses 2 jogadores conseguiram conquistar os 4 GS no mesmo ano, e a ultima vez que aconteceu foi em 1969. Jogadores hoje em dia acham impossível tal feito.

Obs. Aqui foi util ter criado uma tabela de calendario para não ter que ficar criando funções para extrair o ano das datas, é uma boa pratica. 

In [0]:
%sql
SELECT 
  p.player_name,
  c.ano
FROM business.f_matches m
JOIN business.d_tournaments t 
  ON m.tourney_id = t.tourney_id
JOIN business.d_players p 
  ON m.winner_id = p.player_id
JOIN business.d_calendario c
  ON m.tourney_date = c.id_data
WHERE t.tourney_level = 'G'
  AND m.round = 'F'
GROUP BY p.player_name, c.ano
HAVING COUNT(DISTINCT t.tourney_name) >= 4
ORDER BY ano DESC;


player_name,ano
Rod Laver,1969
Rod Laver,1962
Don Budge,1938


### Pergunta 3
- Que jogador permaneceu mais tempo no top 1? E no top 5?

R: O jogador que permaneceu por mais semanas no top 1 foi o Djokovic, porém o jogador que permaneceu mais tempo no top 5 foi o Federer.

Obs.: Essa pergunta foi um pouco mais simples de responder, porém o resultado foi interessante.

In [0]:
%sql
SELECT 
  p.player_name,
  COUNT(*) AS semanas_no_top1
FROM business.d_ranking r
JOIN business.d_players p 
  ON r.player = p.player_id
WHERE r.rank = 1
GROUP BY p.player_name
ORDER BY semanas_no_top1 DESC
LIMIT 10;


player_name,semanas_no_top1
Novak Djokovic,377
Roger Federer,307
Pete Sampras,285
Ivan Lendl,228
Rafael Nadal,193
John McEnroe,136
Jimmy Connors,104
Andre Agassi,100
Bjorn Borg,94
Lleyton Hewitt,80


In [0]:
%sql
SELECT 
  p.player_name,
  COUNT(*) AS semanas_no_top5
FROM business.d_ranking r
JOIN business.d_players p 
  ON r.player = p.player_id
WHERE r.rank <= 5
GROUP BY p.player_name
ORDER BY semanas_no_top5 DESC
LIMIT 10;


player_name,semanas_no_top5
Roger Federer,809
Rafael Nadal,776
Novak Djokovic,721
Pete Sampras,509
Ivan Lendl,478
Boris Becker,441
Jimmy Connors,434
Andre Agassi,431
Stefan Edberg,397
Andy Murray,393


### Pergunta 4
- Que caracteristicas em comum tem essses jogadores? (que permaneceram mais tempo no top 1)

Essa pergunta é um pouco mais abrangente, resolvi deixar ela como um desafio para pensar em como fazer no final, como não me organizei a tempo para explorar ela a fundo, fiz essa análise um pouco mais simples agrupando os jogadores por mão dominante e pais, para ver quantos que atingiram o top 1 possuem cada caracteristica.

R: Maioria dos jogadores que atingiram o top 1 são Americanos (USA) e destros, e a altura média desses 5 jogadores é 185,3, e quando a gente olha para o top 5 o resultado é similar. Mostrando que uma dominancia historica de jogadores americanos.

In [0]:
%sql
SELECT 
  p.hand,
  ROUND(AVG(p.height), 1) AS altura_media,
  p.ioc,
  COUNT(DISTINCT r.player) AS qtde_jogadores
FROM business.d_ranking r
JOIN business.d_players p ON r.player = p.player_id
WHERE r.rank = 1
GROUP BY p.hand, p.ioc
ORDER BY qtde_jogadores DESC;


hand,altura_media,ioc,qtde_jogadores
R,185.3,USA,5
R,180.1,AUS,3
R,194.7,RUS,3
R,184.9,ESP,3
R,183.4,SWE,3
L,179.1,USA,2
L,180.0,AUT,1
R,188.0,SRB,1
R,185.0,SUI,1
R,190.0,GER,1


In [0]:
%sql
SELECT 
  p.hand,
  ROUND(AVG(p.height), 1) AS altura_media,
  p.ioc,
  COUNT(DISTINCT r.player) AS qtde_jogadores
FROM business.d_ranking r
JOIN business.d_players p ON r.player = p.player_id
WHERE r.rank <= 5
GROUP BY p.hand, p.ioc
ORDER BY qtde_jogadores DESC;

hand,altura_media,ioc,qtde_jogadores
R,184.0,USA,19
R,185.2,SWE,8
R,183.0,ESP,7
R,191.9,GER,6
R,181.9,AUS,5
R,190.0,RUS,5
R,186.4,ARG,5
R,192.0,FRA,4
L,179.0,USA,3
R,195.0,CZE,3


## Autoavaliação

Durante o desenvolvimento deste trabalho, acredito que consegui alcançar os principais objetivos que foram definidos no início. O foco foi aplicar conceitos de engenharia de dados a um conjunto de dados do esporte tênis, organizando essas informações de forma estruturada e confiável para facilitar análises.

Consegui organizar os dados em camadas Raw, Trusted e Business. Consegui implementar uma modelagem dimensional utilizando o Esquema Estrela, o que possibilitou análises mais eficientes por meio de tabelas fato e dimensão. Também produzi um dicionário de dados e desenvolvi consultas SQL complexas que ajudaram a responder as questões levantadas no objetivo.

Como foquei na costrução das tabelas utilizando SQL, entre os principais desafios estão: entendimento de subquerys, criação de tabelas a partir das camadas anteriores, tratamento de dados com SQL e toda a nomenclatura em geral da linguagem SQL. Também enfrentei dificuldades técnicas, principalmente no tratamento de dados onde foi dificil passar por todos os campos que precisariam ser tratados de forma pratica.

Para os próximos passos, vejo algumas possibilidades. Uma ideia é de expanção seria incluir dados de partidas de duplas do ranking WTP (feminino) e estatísticas mais detalhadas por set ou ponto. Também penso em integrar o projeto com dados externos, como premiações por torneio e informações climáticas, que podem tornar as análises ainda mais ricas.

Outra possibilidade interessante é a criação de painéis no Power BI, como já tenho experiencia neste assunto, resolvi não evoluir para esse lado neste trabalho. Além disso, seria legal aplicar algoritmos de machine learning para tentar prever os resultados das partidas com base no histórico dos jogadores.

Em resumo, o projeto atingiu as expectativas iniciais e abriu várias oportunidades para aprofundar os conhecimentos em engenharia de dados, além de contribuir para meu portfólio com uma aplicação prática e interessante ligada ao esporte que eu gosto.